<a href="https://colab.research.google.com/github/sarasantosdev/machine_learning_guia_de_referencia_rapida_estudos/blob/main/machine_learning_guia_referencia_rapida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning - Guia de Referência Rápida:
## Trabalhando com dados estruturados em Python, de Matt Harrison

## Tarefa:

**Criar um modelo preditivo que classifique se um passageiro sobreviveu ou não à catástrofe do Titanic, com base em características individuais e de viagem**

In [1]:
import pandas as pd

In [2]:
# importando dataset do titanic

df = pd.read_csv('titanic.csv')

In [3]:
# lendo as cinco primeiras linhas do dataframe

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# coletando informações sobre: tipo de dados e quantidade de valores nulos por coluna

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# obtendo descrições estatísticas sobre os dados

df.describe().astype(int)

# df.describe().iloc[:, 1:3], use para exibir somente colunas específicas (survived e pclass)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891,891,891,714,891,891,891
mean,446,0,2,29,0,0,32
std,257,0,0,14,1,0,49
min,1,0,1,0,0,0,0
25%,223,0,2,20,0,0,7
50%,446,0,3,28,0,0,14
75%,668,1,3,38,1,0,31
max,891,1,3,80,8,6,512


In [6]:
# verifica a quantidade de valores nulos

df.isnull().sum()

# apesar da função isnull retornar True ou False, o Python os interpreta como 1 e 0, por isso podemos somá-los

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [7]:
# removendo coluna 'Cabine' por não ser relevante para a classificação

df = df.drop('Cabin', axis=1) # axis 1 representa as colunas

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [9]:
# verificando a quantidade de pessoas por gênero (value_counts)

df.Sex.value_counts()

,count
Sex,
male,577
female,314


In [10]:
# verificando a quantidade de pessoas por embarque (value_counts)

df.Embarked.value_counts(dropna=False)

,count
Embarked,
S,644
C,168
Q,77
NaN,2


In [11]:
# removendo colunas de dados categóricos não relevantes

df = df.drop(columns=['Name', 'Ticket', 'Fare'])

In [12]:
# criando colunas dummy

df = pd.get_dummies(df, drop_first=True)

df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch',
       'Sex_male', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [13]:
df = df.drop('PassengerId', axis=1)
df.columns

Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Sex_male', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

In [14]:
# separando variável alvo e característcas

y = df.Survived # y recebe a variável alvo
X = df.drop(columns='Survived') # dataframe sem variável alvo

In [19]:
df.tail(10)

,Survived,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
881,0,3,33.0,0,0,True,False,True
882,0,3,22.0,0,0,False,False,True
883,0,2,28.0,0,0,True,False,True
884,0,3,25.0,0,0,True,False,True
885,0,3,39.0,0,5,False,True,False
886,0,2,27.0,0,0,True,False,True
887,1,1,19.0,0,0,False,False,True
888,0,3,NaN,1,2,False,False,True
889,1,1,26.0,0,0,True,False,False
890,0,3,32.0,0,0,True,True,False


In [16]:
from sklearn import model_selection, impute
from sklearn.experimental import enable_iterative_imputer
# separando em conjunto de treinamento e de teste

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
# realizando a imputação dos dados de acordo com o livro
cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_male', 'Embarked_Q', 'Embarked_S']

imputer = impute.IterativeImputer(max_iter=10, random_state=42)

imputed = imputer.fit_transform(X_train[cols])

X_train[cols] = imputed.astype(int)

X_train

,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
445,1,4,0,2,1,0,1
650,3,28,0,0,1,0,1
172,3,1,1,1,0,0,1
450,2,36,1,2,1,0,1
314,2,43,1,1,1,0,1
...,...,...,...,...,...,...,...
106,3,21,0,0,0,0,1
270,1,41,0,0,1,0,1
860,3,41,2,0,1,0,1
435,1,14,1,2,0,0,1


In [18]:
# imputando os dados para o conjunto de testes
cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_male', 'Embarked_Q', 'Embarked_S']

imputer = impute.IterativeImputer(max_iter=10, random_state=42)

imputed = imputer.fit_transform(X_test[cols])

X_test[cols] = imputed.astype(int)

X_test

,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
709,3,24,1,1,1,0,0
439,2,31,0,0,1,0,1
840,3,20,0,0,1,0,1
720,2,6,0,1,0,0,1
39,3,14,1,0,0,0,0
...,...,...,...,...,...,...,...
821,3,27,0,0,1,0,1
633,1,40,0,0,1,0,1
456,1,65,0,0,1,0,1
500,3,17,0,0,1,0,1


In [24]:
# normalize os dados - train
from sklearn import preprocessing

cols = "Pclass, Age, SibSp, Parch, Sex_male, Embarked_Q, Embarked_S".split(",")
sca =  preprocessing.StandardScaler()

X_train = sca.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=cols)

X_train

,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
0,-1.637881,-1.781396,-0.474161,1.998853,0.720772,-0.311177,0.602693
1,0.803267,-0.040632,-0.474161,-0.479327,0.720772,-0.311177,0.602693
2,0.803267,-1.998992,0.348687,0.759763,-1.387401,-0.311177,0.602693
3,-0.417307,0.539623,0.348687,1.998853,0.720772,-0.311177,0.602693
4,-0.417307,1.047346,0.348687,0.759763,0.720772,-0.311177,0.602693
...,...,...,...,...,...,...,...
618,0.803267,-0.548355,-0.474161,-0.479327,-1.387401,-0.311177,0.602693
619,-1.637881,0.902282,-0.474161,-0.479327,0.720772,-0.311177,0.602693
620,0.803267,0.902282,1.171535,-0.479327,0.720772,-0.311177,0.602693
621,-1.637881,-1.056078,0.348687,1.998853,-1.387401,-0.311177,0.602693


In [25]:
# normalize os dados - test

X_test = sca.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns=cols)

X_test

,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
0,0.886122,-0.510516,0.786696,0.786166,0.777683,-0.29905,-1.519422
1,-0.266697,0.021260,-0.522835,-0.460535,0.777683,-0.29905,0.658145
2,0.886122,-0.814388,-0.522835,-0.460535,0.777683,-0.29905,0.658145
3,-0.266697,-1.877940,-0.522835,0.786166,-1.285871,-0.29905,0.658145
4,0.886122,-1.270196,0.786696,-0.460535,-1.285871,-0.29905,-1.519422
...,...,...,...,...,...,...,...
263,0.886122,-0.282612,-0.522835,-0.460535,0.777683,-0.29905,0.658145
264,-1.419516,0.704972,-0.522835,-0.460535,0.777683,-0.29905,0.658145
265,-1.419516,2.604171,-0.522835,-0.460535,0.777683,-0.29905,0.658145
266,0.886122,-1.042292,-0.522835,-0.460535,0.777683,-0.29905,0.658145
